# ĐỒ ÁN MÔN HỌC: DỰ ĐOÁN KẾT QUẢ TENNIS ATP VỚI HỆ THỐNG HYBRID ELO

## 0. Cấu hình môi trường Google Colab

In [ ]:
!git clone https://github.com/chtr302/atp_match_prediction

# Di chuyển vào thư mục dự án vừa tải về
%cd {REPO_NAME}

# Cài đặt các thư viện cần thiết
!pip install pandas numpy scikit-learn matplotlib seaborn xgboost

print("Môi trường Colab đã được cấu hình.")

## 1. Khởi tạo Thư viện và Cấu hình
Import các thư viện cần thiết và thiết lập cấu hình chung cho Notebook.

In [ ]:
import pandas as pd
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier # Thêm MLPClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score, log_loss, roc_curve, auc, confusion_matrix, ConfusionMatrixDisplay

# Cấu hình hiển thị
sns.set_theme(style="whitegrid")
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')

# Thiết lập đường dẫn dữ liệu (tương đối so với thư mục gốc của repo)
DATA_PATH = 'data'
PLOTS_PATH = 'plots'
if not os.path.exists(PLOTS_PATH):
    os.makedirs(PLOTS_PATH)

print("Thư viện đã được import và cấu hình ban đầu hoàn tất.")

## 2. Hệ thống Hybrid Elo tự xây dựng (Chi tiết từ `src/model/custom_elo_model.py`)
Đây là thuật toán tính điểm sức mạnh động (dynamic rating) dựa trên cả phong độ tổng quát và mặt sân, được sử dụng như một đặc trưng (feature) quan trọng cho mô hình học máy.

In [ ]:
class TennisEloModel:
    def __init__(self, k_factor=20, surface_weight=0.5, start_elo=1500):
        """
        Khởi tạo mô hình Elo.
        Args:
            k_factor (int): Hệ số K - độ nhạy của điểm Elo (K càng lớn, điểm càng biến động mạnh).
            surface_weight (float): Trọng số của Elo mặt sân (0.0 đến 1.0).
            start_elo (int): Điểm khởi đầu cho người mới.
        """
        self.k = k_factor
        self.alpha = surface_weight # Trọng số cho surface_elo
        self.overall_elo = {}
        self.surface_elo = {}
        self.start_elo = start_elo

    def get_elo(self, player_id, surface):
        ""Lấy điểm Elo hiện tại của cầu thủ (kết hợp Overall và Surface).""
        # 1. Lấy Overall Elo
        overall = self.overall_elo.get(player_id, self.start_elo)
        
        # 2. Lấy Surface Elo
        if player_id not in self.surface_elo: self.surface_elo[player_id] = {}
        s_elo = self.surface_elo[player_id].get(surface, self.start_elo)
        
        # 3. Tính điểm kết hợp (Weighted Average)
        final_elo = (1 - self.alpha) * overall + self.alpha * s_elo
        return final_elo, overall, s_elo

    def update(self, winner_id, loser_id, surface):
        ""Cập nhật điểm Elo sau mỗi trận đấu.""
        # Lấy điểm hiện tại của người thắng và người thua
        w_final, w_over, w_surf = self.get_elo(winner_id, surface)
        l_final, l_over, l_surf = self.get_elo(loser_id, surface)
        
        # Tính xác suất thắng dự kiến của người thắng (Expected Probability)
        prob_w = 1 / (1 + 10 ** ((l_final - w_final) / 400))
        
        # Tính lượng điểm thay đổi (Delta)
        delta = self.k * (1 - prob_w)
        
        # Cập nhật Overall Elo
        self.overall_elo[winner_id] = w_over + delta
        self.overall_elo[loser_id] = l_over - delta
        
        # Cập nhật Surface Elo
        self.surface_elo.setdefault(winner_id, {})[surface] = w_surf + delta
        self.surface_elo.setdefault(loser_id, {})[surface] = l_surf - delta

    def fit_transform(self, df):
        """
        Chạy mô hình trên toàn bộ dữ liệu lịch sử để sinh ra feature Elo.
        QUAN TRỌNG: Dữ liệu phải được sắp xếp theo thời gian trước!
        """
        
        # Đảm bảo dữ liệu đã sort theo thời gian
        if 'tourney_date' in df.columns:
            df = df.sort_values(by=['tourney_date', 'match_num']).reset_index(drop=True)
            
        elo_diffs = []
        p1_elos = []
        p2_elos = []
        
        # Duyệt qua từng trận đấu theo thứ tự thời gian
        for idx, row in df.iterrows():
            p1_id = row['p1_id']
            p2_id = row['p2_id']
            surface = row['surface']
            target = row['target'] # 1 nếu P1 thắng, 0 nếu P2 thắng
            
            # Lấy Elo TRƯỚC trận đấu (để làm feature dự đoán)
            p1_final_elo, _, _ = self.get_elo(p1_id, surface)
            p2_final_elo, _, _ = self.get_elo(p2_id, surface)
            
            p1_elos.append(p1_final_elo)
            p2_elos.append(p2_final_elo)
            elo_diffs.append(p1_final_elo - p2_final_elo)
            
            # Cập nhật Elo SAU trận đấu (cho các trận tiếp theo)
            if target == 1: # P1 thắng
                self.update(p1_id, p2_id, surface)
            else: # P2 thắng
                self.update(p2_id, p1_id, surface)
                
        # Thêm các cột Elo mới vào DataFrame
        df_new = df.copy()
        df_new['p1_elo'] = p1_elos
        df_new['p2_elo'] = p2_elos
        df_new['elo_diff'] = elo_diffs
        
        return df_new

    # --- 4. Huấn luyện Mô hình Ensemble (Soft Voting) ---
    print("
## 4. Huấn luyện Mô hình Ensemble (Soft Voting)")
    print("Sử dụng kỹ thuật Soft Voting để kết hợp các dự đoán từ Logistic Regression, Random Forest, XGBoost và SVM nhằm đạt hiệu suất tối ưu và ổn định.")

    # --- 4.1. Chia tập dữ liệu (Time-series Split) ---
    # Sắp xếp theo thời gian (đã được thực hiện trong pipeline nhưng kiểm tra lại)
    df_model_ready = df_final_processed.sort_values('tourney_date', ascending=True).reset_index(drop=True)
    
    train_size = int(len(df_model_ready) * 0.8)
    
    # Chọn Features và Target
    X = df_model_ready.drop(columns=['target', 'tourney_date'])
    y = df_model_ready['target'].astype(int)

    X_train, X_test = X.iloc[:train_size], X.iloc[train_size:]
    y_train, y_test = y.iloc[:train_size], y.iloc[train_size:]
    
    print(f"\nKích thước tập Train: {X_train.shape}")
    print(f"Kích thước tập Test : {X_test.shape}")
    
    # --- 4.2. Scale Features ---
    scaler = StandardScaler()
    X_train_s = scaler.fit_transform(X_train)
    X_test_s = scaler.transform(X_test)
    
    # --- 4.3. Định nghĩa và Huấn luyện các Mô hình cơ sở ---
    print("\n--- Đang huấn luyện các Mô hình cơ sở ---")
    # Các mô hình được cấu hình để làm việc tốt với VotingClassifier
    models = [
        ('lr', LogisticRegression(random_state=42, class_weight='balanced', max_iter=1000)),
        ('rf', RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42, class_weight='balanced')),
        ('xgb', XGBClassifier(n_estimators=100, max_depth=5, random_state=42, eval_metric='logloss')),
        ('svm', SVC(probability=True, random_state=42, class_weight='balanced'))
    ]
    
    # --- 4.4. Huấn luyện Voting Ensemble ---
    print("\n--- Đang huấn luyện Voting Ensemble ---")
    voting_clf = VotingClassifier(estimators=models, voting='soft', n_jobs=-1)
    voting_clf.fit(X_train_s, y_train)
    
    # --- 4.5. Đánh giá Mô hình ---
    y_pred = voting_clf.predict(X_test_s)
    y_prob = voting_clf.predict_proba(X_test_s)[:, 1]
    
    acc = accuracy_score(y_test, y_pred)
    loss = log_loss(y_test, y_prob)
    auc_score = roc_auc_score(y_test, y_prob)
    
    print(f"\n--- KẾT QUẢ SOFT VOTING ---")
    print(f"Accuracy: {acc:.4f}")
    print(f"Log Loss: {loss:.4f}")
    print(f"ROC-AUC : {auc_score:.4f}")
else:
    print("Không thể huấn luyện mô hình vì dữ liệu chưa được xử lý thành công.")

## 3. Tiền xử lý dữ liệu hoàn chỉnh (Data Preprocessing Pipeline)
Pipeline này thực hiện các bước từ tải dữ liệu thô, tạo các đặc trưng về phong độ, tái cấu trúc chống rò rỉ dữ liệu (Anti-Leakage) đến tích hợp hệ thống Elo và làm sạch cuối cùng.

In [ ]:
def process_data_pipeline(data_path=DATA_PATH):
    print("\n--- 1. TẢI DỮ LIỆU THÔ VÀ SẮP XẾP THEO THỜI GIAN ---")
    path_pattern = os.path.join(data_path, 'atp_matches_*.csv')
    all_files = glob.glob(path_pattern)
    if not all_files:
        print(f"Lỗi: Không tìm thấy file CSV tại: {data_path}")
        return None
    
    df = pd.concat([pd.read_csv(f) for f in all_files], ignore_index=True)
    df['tourney_date'] = pd.to_datetime(df['tourney_date'], format='%Y%m%d')
    df = df.sort_values(by=['tourney_date', 'match_num']).reset_index(drop=True)
    print(f"Tổng số trận đấu gốc: {len(df)}")

    print("\n--- 2. TÍNH TOÁN ROLLING STATS (PHONG ĐỘ & HIỆU SUẤT MẶT SÂN) ---")
    last_5_stats, surface_stats = {}, {}
    w_forms, l_forms, w_surfs, l_surfs = [], [], [], []
    
    for _, row in df.iterrows():
        wid, lid, surf = row['winner_id'], row['loser_id'], row['surface']
        
        w_forms.append(np.mean(last_5_stats.get(wid, [0])))
        l_forms.append(np.mean(last_5_stats.get(lid, [0])))
        
        ws = surface_stats.get(wid, {}).get(surf, [0, 0])
        w_surfs.append(ws[0]/ws[1] if ws[1]>0 else 0)
        ls = surface_stats.get(lid, {}).get(surf, [0, 0])
        l_surfs.append(ls[0]/ls[1] if ls[1]>0 else 0)
        
        last_5_stats.setdefault(wid, []).append(1)
        if len(last_5_stats[wid]) > 5: last_5_stats[wid].pop(0)
        last_5_stats.setdefault(lid, []).append(0)
        if len(last_5_stats[lid]) > 5: last_5_stats[lid].pop(0)
        
        surface_stats.setdefault(wid, {}).setdefault(surf, [0, 0])
        surface_stats[wid][surf][0] += 1; surface_stats[wid][surf][1] += 1
        surface_stats.setdefault(lid, {}).setdefault(surf, [0, 0])
        surface_stats[lid][surf][1] += 1
        
    df['winner_form'] = w_forms; df['loser_form'] = l_forms
    df['winner_surf'] = w_surfs; df['loser_surf'] = l_surfs

    print("\n--- 3. TÁI CẤU TRÚC (CHỐNG DATA LEAKAGE) VÀ CHỌN FEATURE CƠ BẢN ---")
    # Chỉ chọn các cột có sẵn TRƯỚC trận đấu, thêm các feature mới tạo
    common_cols = ['tourney_id', 'tourney_name', 'surface', 'draw_size', 'tourney_level', 'tourney_date', 'match_num', 'best_of', 'round']
    # Các đặc trưng của người chơi, bao gồm cả 'form' và 'surf' vừa tạo
    p_feats = ['id', 'rank', 'age', 'ht', 'form', 'surf', 'seed'] 
    
    # Ngẫu nhiên hóa vai trò P1/P2
    np.random.seed(42)
    swap = np.random.rand(len(df)) < 0.5
    new_df = pd.DataFrame({'tourney_date': df['tourney_date'], 'surface': df['surface']})
    
    for f in p_feats:
        w_col, l_col = f'winner_{f}', f'loser_{f}'
        if w_col in df.columns:
            new_df[f'p1_{f}'] = np.where(swap, df[l_col], df[w_col])
            new_df[f'p2_{f}'] = np.where(swap, df[w_col], df[l_col])
            
    new_df['target'] = np.where(swap, 0, 1)

    print("\n--- 4. TÍNH TOÁN VÀ TÍCH HỢP ĐIỂM ELO HYBRID ---")
    elo_model = TennisEloModel(k_factor=20, surface_weight=0.5)
    new_df = elo_model.fit_transform(new_df)

    print("\n--- 5. LÀM SẠCH VÀ TẠO FEATURE CHÊNH LỆCH CUỐI CÙNG ---")
    # Xóa hàng thiếu dữ liệu cốt lõi
    new_df.dropna(subset=['p1_rank', 'p2_rank', 'p1_ht', 'p2_ht', 'p1_age', 'p2_age'], inplace=True)
    
    # Điền giá trị cho Seed (Hạt giống): NaN -> 100
    new_df['p1_seed'] = pd.to_numeric(new_df['p1_seed'], errors='coerce').fillna(100)
    new_df['p2_seed'] = pd.to_numeric(new_df['p2_seed'], errors='coerce').fillna(100)

    # Tạo các cột hiệu số (Difference Features)
    new_df['rank_diff'] = new_df['p1_rank'] - new_df['p2_rank']
    new_df['age_diff'] = new_df['p1_age'] - new_df['p2_age']
    new_df['ht_diff'] = new_df['p1_ht'] - new_df['p2_ht']
    new_df['form_diff'] = new_df['p1_form'] - new_df['p2_form']
    new_df['surf_diff'] = new_df['p1_surf'] - new_df['p2_surf']
    
    # One-hot encoding cho 'surface'
    new_df = pd.get_dummies(new_df, columns=['surface'], drop_first=True, prefix='surf')
    
    # Chọn các feature cuối cùng để đưa vào model
    final_features = [col for col in new_df.columns if 'p1_' in col or 'p2_' in col or '_diff' in col or 'surf_' in col or col == 'target']
    # Loại bỏ các cột ID, name, entry, ioc không dùng trực tiếp trong model
    final_features = [f for f in final_features if not any(x in f for x in ['_id', '_name', '_entry', '_ioc'])]
    
    final_df = new_df[final_features].copy()

    return final_df

# --- THỰC THI PIPELINE XỬ LÝ DỮ LIỆU ---
try:
    df_final_processed = process_data_pipeline(data_path=DATA_PATH)
    if df_final_processed is not None:
        # Lưu tạm DataFrame đã xử lý
        df_final_processed.to_csv('processed_atp_data.csv', index=False)
    else:
        pass
except Exception as e:
    df_final_processed = None

## 5. Trực quan hóa Kết quả (Dùng cho Báo cáo)
Phần này tạo ra các biểu đồ quan trọng để minh họa cho báo cáo của bạn.

In [ ]:
if df_final_processed is not None:
    # --- 5.1. ROC Curve ---
    print("\n--- Vẽ biểu đồ ROC Curve ---")
    fpr, tpr, _ = roc_curve(y_test, y_prob)
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, label=f'Soft Voting (AUC={auc_score:.2f})', color='darkorange', lw=2)
    plt.plot([0, 1], [0, 1], 'k--', label='Random Guess (AUC = 0.50)')
    plt.title('ROC Curve for Soft Voting Ensemble')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend(loc='lower right')
    plt.show()
    
    # --- 5.2. Feature Importance (từ thành phần Random Forest) ---
    print("\n--- Vẽ biểu đồ Feature Importance ---")
    rf_estimator = None
    for name, est in voting_clf.estimators:
        if name == 'rf':
            rf_estimator = est
            break
            
    if rf_estimator and hasattr(rf_estimator, 'feature_importances_'):
        imps = rf_estimator.feature_importances_
        feature_names = X_train.columns
        sorted_idx = imps.argsort()[::-1] # Sắp xếp từ cao đến thấp
        
        plt.figure(figsize=(10, 6))
        sns.barplot(x=imps[sorted_idx][:15], y=feature_names[sorted_idx][:15], palette='viridis')
        plt.title('Feature Importance (Random Forest Component in Ensemble)')
        plt.xlabel('Importance')
        plt.ylabel('Feature')
        plt.show()
    else:
        pass

    # --- 5.3. Confusion Matrix ---
    print("\n--- Vẽ biểu đồ Confusion Matrix ---")
    plt.figure(figsize=(8, 6))
    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['P2 Wins', 'P1 Wins'])
    disp.plot(cmap='Blues', values_format='d')
    plt.title('Confusion Matrix for Soft Voting Ensemble')
    plt.show()
else:
    pass